In [1]:
import shutil
import os
import numpy as np
import pandas as pd
import json
from m3inference import M3Inference
from m3inference import M3Twitter
import pprint
from langdetect import detect
import time
m3 = M3Inference()

10/11/2023 23:08:59 - INFO - m3inference.m3inference -   Version 1.1.5
10/11/2023 23:08:59 - INFO - m3inference.m3inference -   Running on cuda.
10/11/2023 23:08:59 - INFO - m3inference.m3inference -   Will use full M3 model.
10/11/2023 23:09:00 - INFO - m3inference.m3inference -   Model full_model exists at /g100/home/userexternal/mhabibi0/m3/models/full_model.mdl.
10/11/2023 23:09:00 - INFO - m3inference.utils -   Checking MD5 for model full_model at /g100/home/userexternal/mhabibi0/m3/models/full_model.mdl
10/11/2023 23:09:00 - INFO - m3inference.utils -   MD5s match.
10/11/2023 23:09:13 - INFO - m3inference.m3inference -   Loaded pretrained weight at /g100/home/userexternal/mhabibi0/m3/models/full_model.mdl


In [2]:
home_dir = '/g100/home/userexternal/mhabibi0/'
work_dir = '/g100_work/IscrC_mental'

hdata_dir = os.path.join(home_dir, 'Data')
wdata_dir = os.path.join(work_dir, 'data')
uc_dir = os.path.join(wdata_dir, 'user_classification')
image_dir = os.path.join(uc_dir, 'images', 'test')
himage_dir =os.path.join(hdata_dir, 'images', 'test')

In [3]:
# # copy images to home

# himage_dir =os.path.join(hdata_dir, 'images', 'test')
# shutil.copytree(image_dir, himage_dir)


In [4]:
# # resize images 
# %cd /g100/home/userexternal/mhabibi0/Data/images/

In [5]:
#!python preprocess.py --source_dir /g100/home/userexternal/mhabibi0/Data/images/test/ --output_dir /g100/home/userexternal/mhabibi0/Data/images/test/images_resized_v2/  

In [6]:
imgres_dir = os.path.join(himage_dir, 'images_resized')

In [7]:
# user age data
path  = os.path.join(uc_dir, 'data_for_models_test.pkl')
df_test = pd.read_pickle(path)

In [8]:
# bio lang
def detect_lang(bio_text):
    # languages that M3 supports
    list_langs = ['en', 'cs', 'fr', 'nl', 'ar', 'ro', 'bs', 'da', 'it', 'pt', 'no',
                  'es', 'hr', 'tr', 'de', 'fi', 'el', 'ru', 'bg', 'hu', 'sk', 'et', 
                  'pl', 'lv', 'sl', 'lt', 'ga', 'eu', 'mt', 'cy', 'rm', 'is', 'un']

    try:
        lang = detect(bio_text)
    except Exception as e:
        lang = 'it'  # default to 'it' in case of exceptions

    # if not supported by M3
    if lang not in list_langs:
        lang = 'it'
    
    return lang

In [9]:
# transform the data to match M3 standard input
img_dir = os.path.join(himage_dir, 'images_resized')
# path to image
df_m3 = df_test[['user_id', 'masked_bio']]
df_m3.columns = ['id', 'description']
df_m3['id'] = df_m3['id'].astype(str)
df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))

# check if processed image exists
id_image_paths = df_m3['img_path'].values.tolist()
nonexist_paths_resized = [path for path in id_image_paths if not os.path.exists(path)]
nonexisting_ids = [path.split('/')[-1].split('.jpeg')[0] for path in nonexist_paths_resized]

# filter to ids having image
df_m3 = df_m3[~ df_m3['id'].isin(nonexisting_ids)]

# bio language
df_m3['lang'] = df_m3['description'].map(detect_lang)

df_m3['name'] = ''
df_m3['screen_name'] = ''

data_json = df_m3.to_dict('records')

/scratch_local/slurm_job.10955448/ipykernel_3813088/1080182451.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['id'] = df_m3['id'].astype(str)
/scratch_local/slurm_job.10955448/ipykernel_3813088/1080182451.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['img_path'] = df_m3['id'].apply(lambda x: os.path.join(img_dir, str(x)+'.jpeg'))


In [10]:
len(data_json)


1107

In [11]:
# from PIL import Image
# import matplotlib.pyplot as plt

# # Load the image
# uid = nonexist_ids[7]+'.jpg'
# image_path = os.path.join(himage_dir, uid)
# img = Image.open(image_path)

# # Display the image
# plt.imshow(img)
# plt.axis('off')  # To turn off axis numbers and ticks
# plt.show()

In [12]:
# find the results
from collections import OrderedDict

batch_size = 32  

results = OrderedDict()

# Process data_json in batches
for i in range(0, len(data_json), batch_size):
    batch = data_json[i:i+batch_size]
    result_batch = m3.infer(batch)
    results.update(result_batch)


10/11/2023 23:09:41 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:05<00:00,  2.60s/it]
10/11/2023 23:09:47 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.14it/s]
10/11/2023 23:09:47 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.57it/s]
10/11/2023 23:09:48 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  3.89it/s]
10/11/2023 23:09:48 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  4.04it/s]
10/11/2023 23:09:49 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  4.04it/s]
10/11/2023 23:09:49 - INFO - m3inference.dataset -   32 data entries loaded.
Predicting...: 100%|██████████| 2/2 [00:00<00:00,  4.27it/s]
10/11/2023 23:09:50 - INFO - m3inf

In [13]:
# save the results

path = os.path.join(uc_dir, 'M3_results_test_bio_image.json')
with open(path, 'w') as f:
    json.dump(results, f)

In [19]:
len(results.keys())

1107

## Bio only 

In [24]:
# transform the data to match M3 standard input

df_m3 = df_test[['user_id', 'masked_bio']]
df_m3.columns = ['id', 'description']
df_m3['id'] = df_m3['id'].astype(str)

# remove cases with no bios
df_m3 = df_m3[ df_m3['description']!='']
# bio language
df_m3['lang'] = df_m3['description'].map(detect_lang)

df_m3['name'] = ''
df_m3['screen_name'] = ''

data_json = df_m3.to_dict('records')

/scratch_local/slurm_job.10950248/ipykernel_14161/762466409.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m3['id'] = df_m3['id'].astype(str)


In [26]:
# Process data_json in batches
results_bio_only = OrderedDict()
m3 = M3Inference(use_full_model=False)
for i in range(0, len(data_json), batch_size):
    batch = data_json[i:i+batch_size]
    result_batch = m3.infer(batch)
    results_bio_only.update(result_batch)

10/11/2023 20:15:32 - INFO - m3inference.m3inference -   Version 1.1.5
10/11/2023 20:15:32 - INFO - m3inference.m3inference -   Running on cuda.
10/11/2023 20:15:32 - INFO - m3inference.m3inference -   Will use text model. Note that as M3 was optimized to work well with both image and text data,                                     it is not recommended to use text only model unless you do not have the profile image.
10/11/2023 20:15:32 - INFO - m3inference.m3inference -   Model text_model does not exist at /g100/home/userexternal/mhabibi0/m3/models/text_model.mdl. Try to download it now.
10/11/2023 20:15:32 - INFO - m3inference.utils -   /g100/home/userexternal/mhabibi0/m3/models/text_model.mdl not found in cache, downloading from https://nlp.stanford.edu/~zijwang/m3inference/text_model.mdl to /scratch_local/slurm_job.10950248/tmpry9dsq2f
31589KB [00:05, 5504.71KB/s]                            
10/11/2023 20:15:39 - INFO - m3inference.utils -   Model text_model was downloaded to a tmp 

In [27]:
list_keys = list(results_bio_only.keys())

In [31]:
# save the results
path = os.path.join(uc_dir, 'M3_results_test_bio_only.json')
with open(path, 'w') as f:
    json.dump(results_bio_only, f)